### CryoParticleSegment

In [ ]:
%pip install torchinfo -qq
%pip install -U git+https://github.com/qubvel/segmentation_models.pytorch -qq
%pip install https://github.com/soft-matter/trackpy/archive/master.zip -qq
%pip install pycuda -qq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 33.1 MB/s eta 0:00:00
     \ 10.7 MB 9.1 MB/s 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00


## ⭐ Setup
You must run all codes under this category.

### ✅ Directory Settings

In [ ]:
# @title  { display-mode: "form" }

IMAGE_DIR = "/content/drive/MyDrive/research_xs/dataset/10081/processed_micrographs_np_split" # @param {type:"string"}
LABEL_DIR = "/content/drive/MyDrive/research_xs/dataset/10081/micrographs_ground_np" # @param {type:"string"}
RESULT_DIR = "/content/drive/MyDrive/research_xs/results/10081_test/unet_eb5_dice_postcrf/" # @param {type:"string"}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# @title  { display-mode: "form" }
# @markdown Detect whether using folder in Google Drive as **`RESULT DIR`**📁.

if "content" in IMAGE_DIR.split("/")[:3] or "content" in LABEL_DIR.split("/")[:3]:
  try:
    from google.colab import drive
    drive.mount('/content/drive')
    !rm -r /content/sample_data
    if "content" in IMAGE_DIR.split("/")[:3]:
      !cp -r {IMAGE_DIR} /content/image_dir
      IMAGE_DIR = "/content/image_dir"
    if "content" in LABEL_DIR.split("/")[:3]:
      !cp -r {LABEL_DIR} /content/label_dir
      LABEL_DIR = "/content/label_dir"
  except:
    pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
IMAGE_DIR = "/content/image_dir"

In [ ]:
# @title  { display-mode: "form" }
# @markdown Source code directory.
SRC_DIR = "/content/drive/MyDrive/00_Image\ meeting/Researches/01_Scratchpad/seg_src/CryoParticleSegment-main/Modeling" # @param {type:"string"}

if True:
  !cp -rf {SRC_DIR}/* /content/
else:
  !cp {SRC_DIR}/convcrf.py /content/convcrf.py
  !cp {SRC_DIR}/dataset.py /content/dataset.py
  !cp {SRC_DIR}/lr_scheduler.py /content/lr_scheduler.py
  !cp {SRC_DIR}/metrics.py /content/metrics.py
  !cp {SRC_DIR}/model.py /content/model.py
  !cp {SRC_DIR}/trainer.py /content/trainer.py
  !cp {SRC_DIR}/utils.py /content/utils.py

In [ ]:
%git clone https://github.com/netw0rkf10w/CRF.git
%cd CRF
!python setup.py install

/content/CRF_main
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.


In [ ]:
%cd /content/

/content


### ✅ Packages Handling

In [ ]:
# @title  { display-mode: "form" }
# @markdown Useful packages.

import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau, OneCycleLR

In [ ]:
# @title  { display-mode: "form" }
# @markdown User-defined packages.

from dataset import MicrographDataset, MicrographDatasetEvery
from dataset import reconstruct_patched
from model import create_model
from trainer import CryoEMEvaluator
from trainer import CryoEMTrainerWithScheduler, tqdm_plugin_for_Trainer

## ⭐ Main

### ✅ Setting

In [ ]:
# @markdown Parameters.

NUM_CLASSES = 2
EPOCHS = 50
BATCH = 8
CROP_SIZE = (512, 512)
LR = 1e-3
RLR_PATIENCE = 3
ES_PATIENCE = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# @markdown Set seed.

random_state = 42
torch.manual_seed(random_state)
torch.cuda.manual_seed_all(random_state)

## ⭐ Convcrf wtih FCN finetuned on cryoem

### ✅ Model

## The model

In [ ]:
# @title  { display-mode: "form" }

architecture = "Unet++" # @param {type:"string"}
encoder = "timm-efficientnet-b5" # @param {type:"string"}
pretrained = True # @param {type:"boolean"}
solver = "fw" # @param {type:"string"}
use_unary_only = True # @param {type:"boolean"}


In [ ]:
import segmentation_models_pytorch as smp

if pretrained:
  weights = "imagenet"
else:
  weights = None

if architecture == "Unet++":
    backbone = smp.UnetPlusPlus(
        encoder_name=encoder,        # choose encoder, densenet201, resnet50, e.g. mobilenet_v2 or efficientnet-b5
        encoder_weights=weights,     # use `imagenet` or `advprop` for pre-trained weights for encoder initialization
        in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset)
    )

elif architecture == "Deeplab":
    backbone = smp.DeepLabV3(
        encoder_name=encoder,        # choose encoder, densenet201, resnet50, e.g. mobilenet_v2 or efficientnet-b5
        encoder_weights=weights,     # use `imagenet` pre-trained weights for encoder initialization
        in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
        classes=2,                      # model output channels (number of classes in your dataset)
    )
else:
    print("Architecture not supported")
    raise NotImplementedError

model = create_model(backbone, addout=True) #crf_args

Downloading: "https://github.com/huggingface/pytorch-image-models/releases/download/v0.1-weights/tf_efficientnet_b5-c6949ce9.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnet_b5-c6949ce9.pth
100%|██████████| 117M/117M [00:00<00:00, 369MB/s]


In [ ]:
import CRF
import torch.nn as nn
from model import setup_crf, create_fwcrf_model

# Example usage
solver = 'fw'  # Assuming the solver type is defined

crf = setup_crf(solver, NUM_CLASSES)
model_post = create_fwcrf_model(model.backbone, crf, use_unary_only=True)

CRF solver: fw
x0_weight: 0.0
FrankWolfeParams: 
	 scheme:	 fixed 
	 stepsize:	 1.0 (for the 'fixed' scheme) 
	 regularizer:	 l2
	 lambda_:	 1.0
	 lambda_learnable:	 False
	 x0_weight:	 0.5
	 x0_weight_learnable:	 False
Non-trainable lambda for Frank-Wolfe: 1.0
Non-trainable x0_weight for Frank-Wolfe: 0.5
Potts: remove random weights.
Add 1.0 to spatial_weight diagonal
Add 1.0 to bilateral_weight diagonal
Add -1.0 to compatibility diagonal


## ⭐ Evaluate

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

from torchvision.utils import save_image
from dataset import reconstruct_patched

def simple_micrograph_preprocessing(micrograph):
  micrograph_copy = micrograph.copy()
  micrograph_copy = (micrograph_copy-micrograph.mean()+2.5*micrograph.std())/5/micrograph.std()
  micrograph_copy[micrograph_copy<0]=0
  micrograph_copy[micrograph_copy>1]=1
  return micrograph_copy

In [ ]:
from torch.utils.data import ConcatDataset

train_dir = os.path.join(IMAGE_DIR, 'train')
train_filenames = np.loadtxt(f"{IMAGE_DIR}/train_filenames.txt", dtype=str)
train_dataset =  MicrographDatasetEvery(image_dir=train_dir, label_dir=LABEL_DIR, filenames=train_filenames, crop_size=CROP_SIZE, crop=None)
train_loader = DataLoader(train_dataset , batch_size=None, shuffle=False, pin_memory=True)

val_dir = os.path.join(IMAGE_DIR, 'val')
val_filenames = np.loadtxt(f"{IMAGE_DIR}/val_filenames.txt", dtype=str)
val_dataset = MicrographDatasetEvery(image_dir=val_dir, label_dir=LABEL_DIR, filenames=val_filenames, crop_size=CROP_SIZE, crop=None)
val_loader = DataLoader(val_dataset, batch_size=None, shuffle=False, pin_memory=True)

test_dir = os.path.join(IMAGE_DIR, 'test')
test_filenames = np.loadtxt(f"{IMAGE_DIR}/test_filenames.txt", dtype=str)
test_dataset = MicrographDatasetEvery(image_dir=test_dir, label_dir=LABEL_DIR, filenames=test_filenames, crop_size=CROP_SIZE, crop=None)
test_loader = DataLoader(test_dataset, batch_size=None, shuffle=False, pin_memory=True)

full_filenames = np.concatenate((train_filenames, val_filenames, test_filenames))
full_dataset = ConcatDataset([train_dataset, val_dataset, test_dataset])
full_loader = DataLoader(full_dataset, batch_size=32, shuffle=True, pin_memory=True)

In [ ]:
shape = None
for i1, i2, i3, i4 in val_loader: #test loader and reconstruct
    shape = i4.shape
    break

In [ ]:
from PIL import Image

def normalize(im):
    max_mrc=np.max(im)
    min_mrc=np.min(im)
    img_original=(255*((im-min_mrc)/(max_mrc-min_mrc))).astype(np.uint8)
    return(img_original)

def preprocess_and_crop(micrograph, crop_size=3840):
    processed_micrograph = simple_micrograph_preprocessing(micrograph)
    if crop_size:
        mic_width, mic_height = processed_micrograph.shape[1], processed_micrograph.shape[0]
        start_x, start_y = (mic_width - crop_size) // 2, (mic_height - crop_size) // 2
        end_x, end_y = start_x + crop_size, start_y + crop_size
        return processed_micrograph[start_y:end_y, start_x:end_x]
    else:
        return processed_micrograph

In [ ]:
from tqdm import tqdm

fnames = []
n = len(full_dataset)
processed_micrographs = np.empty((n, shape[1], shape[2]), dtype=np.float32)

# Use tqdm to wrap your loop for a progress bar
for idx, (test_image, _, grid, _) in tqdm(enumerate(full_dataset), total=n, desc="Processing images"):
    name = full_filenames[idx][:-4]
    fnames.append(name)
    # Determine the directory and load the micrograph
    if os.path.exists(f"{IMAGE_DIR}/test/{name}.npy"):
        micrograph_path = f"{IMAGE_DIR}/test/{name}.npy"
    elif os.path.exists(f"{IMAGE_DIR}/train/{name}.npy"):
        micrograph_path = f"{IMAGE_DIR}/train/{name}.npy"
    else:
        micrograph_path = f"{IMAGE_DIR}/val/{name}.npy"
    micrograph = np.load(micrograph_path)
    processed_micrograph = preprocess_and_crop(micrograph, crop_size=None)
    # Place the processed micrograph directly into the preallocated array
    processed_micrographs[idx] = processed_micrograph

Processing images:  48%|████▊     | 144/300 [03:26<04:19,  1.67s/it]

In [ ]:
RESULT_DIR = "/content/drive/MyDrive/research_xs/results/10081_test/unet_eb5_dice_postcrf/" #model3

In [ ]:
processed_micrographs.shape

In [ ]:
np.save(f"{RESULT_DIR}/processed_micrographs.npy", processed_micrographs)

In [ ]:
processed_micrographs = np.load(f"{RESULT_DIR}/processed_micrographs.npy")

In [ ]:
del model
torch.cuda.empty_cache()

In [ ]:
model = model_post

In [ ]:
RESULT_DIR

In [ ]:
import matplotlib.image as mpimg

# Load the image
mic = mpimg.imread('label_dir/Falcon_2012_06_12-15_07_41_0.png')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12))  # Create two subplots side by side
# Plot on the first axis
ax1.imshow(mic, cmap='Greys_r')
ax1.set_title("Original Micrograph")

# Plot on the second axis (Zoomed-in area)
ax2.imshow(mic, cmap='gray')
ax2.set_xlim([2228, 3428])  # Set limits for x-axis
ax2.set_ylim([3328, 2128])  # Set limits for y-axis; inverted for typical image coordinate system
ax2.set_title("Zoomed-in View")

In [ ]:
import torch.nn.functional as F

label_images = np.empty((0, shape[1], shape[2]), dtype=np.uint8)

checkpoint_paths = [path for path in os.listdir(RESULT_DIR) if '.pt' in path]
checkpoint_path = checkpoint_paths[-1]
state_dict_path = f"{RESULT_DIR}/{checkpoint_path}"
state_dict = torch.load(state_dict_path, map_location=torch.device(device))
model.load_state_dict(state_dict, strict=False)

model.to(device)
model.eval()

mini_batch_size = 18  # Number of patches to process at once
n = len(full_dataset)
label_images = np.empty((n, shape[1], shape[2]), dtype=np.uint8)  # Preallocated array

# Iterate through the dataset with tqdm for progress tracking
for idx, (test_image, _, grid, _) in tqdm(enumerate(full_dataset), total=n, desc="Processing dataset"):
    # Process in batches
    with torch.no_grad():
        inputs = test_image.to(device)
        num_batches = (inputs.size(0) + mini_batch_size - 1) // mini_batch_size
        patched_outputs = []

        for batch_idx in range(num_batches):
            start_idx = batch_idx * mini_batch_size
            end_idx = min(start_idx + mini_batch_size, inputs.size(0))
            patch_input = inputs[start_idx:end_idx].to(device)
            output = model(patch_input)['out']
            patched_outputs.append(output.cpu())  # Minimize device memory usage

            # Cleanup as soon as not needed
            del patch_input, output
            torch.cuda.empty_cache()

        outputs = torch.cat(patched_outputs)  # Combine outputs
        probabilities = F.softmax(outputs, dim=1)
        class1_probabilities = probabilities[:, 1, :, :]  # Assuming class 1 is the target
        pred_image = reconstruct_patched(class1_probabilities.unsqueeze(1), grid).float()
        output_image = normalize(pred_image.squeeze().numpy())

        # Cleanup large temporary variables
        del patched_outputs, outputs, probabilities, class1_probabilities, pred_image
        torch.cuda.empty_cache()

    # Store the output image directly in the preallocated array
    label_images[idx] = output_image

    if idx % 30 == 0:
        _, ax = plt.subplots(figsize=(12, 12))
        ax.imshow(processed_micrographs[idx], cmap='gray')
        ax.imshow(output_image, cmap='inferno', alpha=0.4)
        plt.show()
    del output_image
    torch.cuda.empty_cache()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
np.save(f"{RESULT_DIR}/label_images.npy", label_images)

In [ ]:
!cp {RESULT_DIR}/label_images.npy .

In [ ]:
label_images = np.load(f"{RESULT_DIR}/label_images.npy")

In [ ]:
label_images.shape

(300, 3838, 3710)

In [ ]:
# @title  { vertical-output: true, display-mode: "form" }
EMPIAR_ID = 10081 # @param {type:"integer"}
RADIUS = 77 # @param {type:"integer"}
# For 10017
BORDER = 0 # @param {type:"integer"}

### Actual seg

In [ ]:
radius = RADIUS

In [ ]:
import skimage as ski
from skimage import img_as_float
from scipy import ndimage as ndi
from skimage import filters, segmentation, morphology
import itertools
import cv2


def min_rect_circle(cont):
    contours_poly = cv2.approxPolyDP(cont, 3, True)
    center, _= cv2.minEnclosingCircle(contours_poly)
    rect=cv2.minAreaRect(cont)
    box=np.int0(cv2.boxPoints(rect))
    mn,mx=np.amin(box,axis=0),np.amax(box,axis=0)
    diff=mx-mn
    if np.all(diff<(2*radius+40)):
        return(int(center[0]),int(center[1]))
    else:
        pass

def eliminate_near(fields):
    fields=np.array(fields,dtype=np.int32)
    i=np.arange(len(fields))
    diff=fields.reshape(len(fields),1,2)-fields
    D=np.sqrt((diff**2).sum(2))
    D=np.array(D,dtype=np.float64)
    D[np.triu_indices(D.shape[0])]=np.inf
    re = np.where(D< radius)
    out=np.array(list(zip(re[0],re[1])),dtype=np.int32)
    outmin=np.unique(np.min(out,axis=1))
    return(outmin)

def primary_sorting(i):
    #print("primary_sorting done")
    i1,i2,i3,i4=i[1]-(radius*2),i[0]-(radius*2),i[1]+(radius*2),i[0]+(radius*2)
    if i1<0 and i2<0:
        center=(i1+radius*2,i1+radius*2)
        i1=0
        i2=0
    elif i3 > thresh1.shape[0] and i4 > thresh1.shape[1]:
        i3= thresh1.shape[0]
        i4 = thresh1.shape[1]
        center=(radius*2,radius*2)
    elif i1<0:
        center=(i1+radius*2,i1+radius*2)
        i1=0
    elif i2<0:
        center=(i2+radius*2,i2+radius*2)
        i2=0
    elif i3 > thresh1.shape[0]:
        i3= thresh1.shape[0]
        center=(radius*2,radius*2)
    elif i4 > thresh1.shape[1]:
        i4 = thresh1.shape[1]
        center=(radius*2,radius*2)
    else:
        center=(radius*2,radius*2)


    th1=thresh1[i1:i3,i2:i4]

    c_frame=output_image[i1:i3,i2:i4]
    c_frame = cv2.cvtColor(c_frame, cv2.COLOR_GRAY2RGB)
    mask = np.zeros_like(th1)
    mask2 = np.zeros_like(th1)
    mask3 = np.zeros_like(c_frame)
    mask=cv2.circle(mask, center=center, radius=radius,color=(255),thickness=-1)
    mask2=cv2.circle(mask2, center=center, radius=radius+10,color=(255),thickness=-1)
    mask3=cv2.circle(mask3, center=center, radius=radius+10,color=[255,255,255],thickness=-1)
    col_mask= np.bitwise_and(c_frame,mask3)
    result = np.bitwise_and(th1,mask)
    result2 = np.bitwise_and(th1,mask2)
    col_a=np.where(np.all(col_mask== [0,255,0] , axis=-1))
    col_b=np.where(np.all(col_mask== [0,255,255] , axis=-1))
    inn_positions=np.nonzero(result)
    out_positions=np.nonzero(result2)

    contours, hierarchy = cv2.findContours(result,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    l=len(contours)
    a,b= (inn_positions[0].max()-inn_positions[0].min()),(inn_positions[1].max()-inn_positions[1].min())
    c,d= (out_positions[0].max()-out_positions[0].min()),(out_positions[1].max()-out_positions[1].min())
    if (len(col_a[0]) or len(col_b[0]))>0 :
        pass
    elif (c or d) > (radius*2.1):
        pass
    elif l==1 and ((c and d) <(radius*2)):
        return(tuple([i[0],i[1]]))
    else:

        #cv2.circle(frame, i, radius, (255, 0, 0), 3)
        ans={}
        centerx=range(0+radius,th1.shape[1]-radius,5)
        centery=range(0+radius,th1.shape[0]-radius,5)
        rad=range(radius,5,5)
        for cx,cy in itertools.product(centerx,centery):
            mask00 = np.zeros_like(th1)
            mask20=mask.copy()
            mask00=cv2.circle(mask00, center=(cx,cy), radius=radius, color=(255,255,255), thickness=-1)
            mask20=cv2.circle(mask20, center=(cx,cy), radius=radius+5, color=(255,255,255), thickness=-1)
            result00 = np.bitwise_and(th1,mask00)
            tot_ellip=np.sum(mask00 == [255])
            tot_ellip_white1=np.sum(result00 == [255])
            per_white=(tot_ellip_white1/tot_ellip)*100
            result002 = np.bitwise_and(th1,mask20)
            tot_ellip_white2=np.sum(result002 == [255])
            diff=tot_ellip_white2-tot_ellip_white1

            if diff<10 and per_white>10:
                ans[per_white]=(cx,cy)


        if len(ans.keys())>0:
            k=ans[max(ans.keys())]
            return (tuple([i2+k[0],i1+k[1]]))
        else:
            pass

In [ ]:
BORDER

0

In [ ]:
from skimage import img_as_float
from skimage.filters import threshold_local

cv_list = []
for img in label_images:
    output_image = img_as_float(img)
    block_size = int(round(radius * 1.6)) | 1  # Ensure it's an odd number
    local_thresh = filters.threshold_local(output_image, block_size, method='gaussian', offset=0)
    image_seg = output_image > local_thresh
    thresh1 = ski.util.img_as_ubyte(image_seg)

    contr_min = radius*0.6
    kernel_size = int(radius / 2)  # Example ratio
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    thresh1 = cv2.erode(thresh1, kernel, iterations=1)

    contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cont_array = [c for c in contours]

    # Filter out small/large region and find bounding box with center
    c_ = np.array([cv2.contourArea(contour) for contour in contours])
    aa = (c_>contr_min) & (c_<500000)
    aa = aa.tolist()
    c_full_list = [d for d, keep in zip(cont_array, aa) if keep]
    c_list = (list(map(lambda x: min_rect_circle(x), c_full_list)))
    c_list = [x for x in c_list if x is not None]
    cv_list.append(c_list)
    print(len(c_full_list), len(c_list))

<ipython-input-57-7d9b2ef565d5>:13: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box=np.int0(cv2.boxPoints(rect))


94 93
49 49
13 13
144 144
163 159
146 144
152 148
197 190
258 249
185 183
163 158
167 165
153 149
139 136
198 193
223 217
255 239
173 170
144 142
139 136
139 136
193 187
173 172
106 105
97 96
151 146
160 154
152 151
97 96
142 140
98 95
125 125
104 104
279 271
160 156
173 168
113 112
137 136
100 99
115 115
102 102
123 120
121 117
174 171
183 179
92 92
92 91
54 54
98 97
88 88
70 69
18 18
110 110
45 45
31 31
66 66
58 58
28 28
100 100
105 105
57 57
85 85
43 43
75 73
38 38
61 60
76 76
82 82
135 134
115 114
79 78
69 69
75 75
55 53
0 0
3 3
36 36
35 35
116 116
62 61
88 87
35 35
66 66
80 80
88 88
56 55
64 64
155 153
169 168
19 19
35 35
13 13
197 193
162 158
167 165
190 182
147 144
189 188
149 147
170 168
181 181
157 155
105 103
144 144
159 158
144 143
176 173
188 185
166 164
134 133
126 125
172 169
137 135
149 147
126 123
154 153
176 173
168 167
176 175
143 141
164 163
155 152
145 143
201 194
243 236
175 172
200 196
148 144
160 158
154 152
193 193
225 219
230 219
228 224
253 236
217 217
200 191

In [ ]:
import matplotlib
for idx, (test_image, _, grid, _) in enumerate(full_dataset):
    #if idx == 6:
    #    break
    if idx % 30 == 0:
        label_image = label_images[idx]
        _, ax = plt.subplots(figsize=(12, 12))
        ax.imshow(processed_micrographs[idx], cmap='gray')
        ax.imshow(label_image, cmap='inferno', alpha=0.4)

        for indx in cv_list[idx]:
            x, y = indx
            c = matplotlib.patches.Circle((x, y), radius=RADIUS, fill=False, color='r')
            ax.add_patch(c)
        plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install starfile -qq

In [ ]:
!cp /content/drive/MyDrive/research_xs/dataset/{EMPIAR_ID}/ground_truth/empiar-{EMPIAR_ID}_particles_selected.star .

In [ ]:
import starfile
optics, df = starfile.read(f"empiar-{EMPIAR_ID}_particles_selected.star")['optics'], starfile.read(f"empiar-{EMPIAR_ID}_particles_selected.star")['particles']
df

,rlnImageName,rlnMicrographName,rlnCoordinateX,rlnCoordinateY,rlnAnglePsi,rlnOriginXAngst,rlnOriginYAngst,rlnDefocusU,rlnDefocusV,rlnDefocusAngle,rlnPhaseShift,rlnCtfBfactor,rlnOpticsGroup,rlnClassNumber
0,000001@>J16/extract/HCN1apo_0183_2xaligned_par...,>J1/imported/000008714401408334489_HCN1apo_018...,2041,384,349.897949,-0.422500,-1.267500,18693.681641,17792.365234,-75.985649,0.0,0.0,1,38
1,000002@>J16/extract/HCN1apo_0183_2xaligned_par...,>J1/imported/000008714401408334489_HCN1apo_018...,985,2821,103.775505,1.267500,-2.112500,18627.705078,17726.388672,-75.985649,0.0,0.0,1,38
2,000003@>J16/extract/HCN1apo_0183_2xaligned_par...,>J1/imported/000008714401408334489_HCN1apo_018...,2258,2066,57.857143,-0.422500,2.112500,18799.509766,17898.193359,-75.985649,0.0,0.0,1,38
3,000004@>J16/extract/HCN1apo_0183_2xaligned_par...,>J1/imported/000008714401408334489_HCN1apo_018...,1529,620,276.428558,-1.267500,-2.112500,18722.833984,17821.517578,-75.985649,0.0,0.0,1,38
4,000005@>J16/extract/HCN1apo_0183_2xaligned_par...,>J1/imported/000008714401408334489_HCN1apo_018...,3416,256,283.775482,-0.422500,-0.422500,18868.507812,17967.191406,-75.985649,0.0,0.0,1,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39347,000179@>J16/extract/HCN1apo_0686_2xaligned_par...,>J1/imported/018401356561406588198_HCN1apo_068...,1861,134,26.632650,-3.802500,14.787499,21976.574219,20989.371094,-75.429733,0.0,0.0,1,20
39348,000180@>J16/extract/HCN1apo_0686_2xaligned_par...,>J1/imported/018401356561406588198_HCN1apo_068...,966,2712,302.142822,12.252500,-9.717500,22019.349609,21032.146484,-75.429733,0.0,0.0,1,37
39349,000182@>J16/extract/HCN1apo_0686_2xaligned_par...,>J1/imported/018401356561406588198_HCN1apo_068...,2316,2847,173.571426,13.942500,8.027500,21821.414062,20834.210938,-75.429733,0.0,0.0,1,48
39350,000183@>J16/extract/HCN1apo_0686_2xaligned_par...,>J1/imported/018401356561406588198_HCN1apo_068...,2047,1682,112.959175,-11.407499,-14.787499,21797.146484,20809.943359,-75.429733,0.0,0.0,1,46


In [ ]:
import pandas as pd

for idx, fname in enumerate(fnames):
    # Create the adjusted list of tuples for current index
    adjusted_c_list = [(x + BORDER, y + BORDER) for x, y in cv_list[idx]]

    # Create a temporary DataFrame from the list of tuples
    temp_df = pd.DataFrame(adjusted_c_list, columns=['rlnCoordinateX', 'rlnCoordinateY'])
    temp_df['rlnImageName'] = [f"{i:05d}@{fname}.mrc" for i in range(len(temp_df))]
    temp_df['rlnMicrographName'] = fname + ".mrc"

    # If this is the first DataFrame, initialize df
    if idx == 0:
        df = temp_df
    else:
        # Append the temporary DataFrame to the existing df
        df = pd.concat([df, temp_df], ignore_index=True)

In [ ]:
df

,rlnCoordinateX,rlnCoordinateY,rlnImageName,rlnMicrographName
0,729,3777,00000@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
1,1421,3764,00001@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
2,517,3690,00002@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
3,2489,3690,00003@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
4,739,3585,00004@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
...,...,...,...,...
43807,604,62,00217@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc
43808,3125,55,00218@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc
43809,1830,60,00219@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc
43810,3361,106,00220@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc


In [ ]:
starfile.write(df, f'{RESULT_DIR}/cv_particles.star')

### TrackPy

In [ ]:
import trackpy as tp

In [ ]:
TrackParticleSize = 153
curr_adpmass = 0
sep = 61
scale = 0.25  # Scale factor (0.5 means reducing the size to half)

In [ ]:
tp_list = []
for img in label_images:
    output_image = img
    small_image = cv2.resize(output_image, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    # Adjust parameters based on the scale
    small_sep = int(sep * scale)
    small_diameter = int(TrackParticleSize * scale)
    if small_diameter % 2 == 0:
        small_diameter += 1
    coorTrack = tp.locate(small_image, diameter=small_diameter, minmass=curr_adpmass, separation=small_sep)
    #coorTrack.loc[:,'prob']=[output_image[getattr(coor,'y'),getattr(coor,'x')] for coor in np.floor(coorTrack[['x','y']]).astype('int').itertuples()]
    coorTrack['x'] *= (1/scale)
    coorTrack['y'] *= (1/scale)
    coords = coorTrack[['x', 'y']].values
    tp_list.append(coords)
    print(len(coords))

90
44
12
142
161
156
160
208
275
195
166
164
153
134
198
232
282
168
143
157
156
187
177
107
98
144
169
154
98
147
98
126
106
292
165
176
114
133
104
117
106
119
130
185
189
92
94
58
96
85
73
23
107
50
31
66
52
36
98
104
61
83
43
70
36
63
75
78
132
120
78
63
74
57
9
6
30
31
113
62
86
35
66
81
89
57
59
152
158
20
30
20
198
170
166
194
149
183
152
180
179
158
111
141
162
144
173
182
157
132
127
169
144
156
128
151
175
166
193
141
164
161
149
206
254
182
202
149
168
157
194
239
250
228
258
238
217
255
184
263
283
262
272
299
280
253
254
245
207
225
206
237
152
207
149
155
157
156
168
146
160
153
131
172
183
157
145
170
217
176
216
232
178
173
176
226
134
184
191
175
216
179
187
188
163
189
201
178
164
181
202
125
105
101
116
140
88
150
129
145
143
85
144
83
101
131
138
144
145
155
206
144
123
160
119
112
115
124
151
153
129
131
129
205
149
141
131
161
112
118
101
154
141
114
123
146
141
139
247
283
244
259
230
272
234
112
88
47
89
75
65
56
108
109
151
154
129
145
184
174
194
184
241
241
1

In [ ]:
import matplotlib
for idx, (test_image, _, grid, _) in enumerate(full_dataset):
    # if idx == 6:
    #     break
    if idx % 30 == 0:
        label_image = label_images[idx]
        _, ax = plt.subplots(figsize=(12, 12))
        ax.imshow(processed_micrographs[idx], cmap='gray')
        ax.imshow(label_image, cmap='inferno', alpha=0.4)

        for indx in tp_list[idx]:
            x, y = indx
            c = matplotlib.patches.Circle((x, y), radius=RADIUS, fill=False, color='r')
            ax.add_patch(c)
        plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd

for idx, fname in enumerate(fnames):
    # Create the adjusted list of tuples for current index
    adjusted_c_list = [(x + BORDER, y + BORDER) for x, y in tp_list[idx]]

    # Create a temporary DataFrame from the list of tuples
    temp_df = pd.DataFrame(adjusted_c_list, columns=['rlnCoordinateX', 'rlnCoordinateY'])
    temp_df['rlnImageName'] = [f"{i:05d}@{fname}.mrc" for i in range(len(temp_df))]
    temp_df['rlnMicrographName'] = fname + ".mrc"

    # If this is the first DataFrame, initialize df
    if idx == 0:
        df = temp_df
    else:
        # Append the temporary DataFrame to the existing df
        df = pd.concat([df, temp_df], ignore_index=True)

In [ ]:
df

,rlnCoordinateX,rlnCoordinateY,rlnImageName,rlnMicrographName
0,1129.319838,106.044009,00000@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
1,429.396330,131.435529,00001@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
2,3231.505852,166.323780,00002@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
3,1205.192377,165.609999,00003@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
4,3469.696527,295.035429,00004@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
...,...,...,...,...
45958,3011.966549,3721.648035,00212@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc
45959,2875.534449,3692.463562,00213@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc
45960,943.195842,3740.767483,00214@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc
45961,410.545901,3762.503888,00215@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc


In [ ]:
import starfile
starfile.write(df, f'{RESULT_DIR}/tp_particles.star')

### Nonmax

In [ ]:
#ratio=1
pnum=1000 #initial filtering, larger if candidate is more
pCan=0.4 #the grid size smaller will generate more candidate
overlap=0.6 #allow for overlap
psize=RADIUS*2

# Nor affect too much
level=3
nSep=10
nIter=100

In [ ]:
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import pycuda.autoinit  # Automatically initializes CUDA, creates a context which is needed
import cv2

mod = SourceModule("""
__global__ void scoreGpu(float* heat,float*gaus, float*score, int sizex, int sizey, int psize)
{
    // 2D Thread ID (assuming that only *one* block will be executed)
    int tx=threadIdx.x;
    int ty=threadIdx.y;

    tx=blockIdx.x*blockDim.x+tx;
    ty=blockIdx.y*blockDim.y+ty;

    int tmp=psize/2;
    if((tx >= tmp) && (ty >= tmp) && (tx <= sizex-tmp) && (ty < sizey-tmp) ){
        int sx=tx-tmp;
        int sy=ty-tmp;
        heat=heat+sx*sizex+sy;
        //heat=heat+tx*sizex+ty;
        float sum=0;
        int sub=0;
        for(int i=0;i<psize;i++){
            for(int j=0;j<psize;j++){
                sum=sum+gaus[sub]*heat[j];
                sub=sub+1;
            }
            heat=heat+sizex;
        }
        score[tx*sizex+ty]=sum;

    }//end if
}
__global__ void getMax(float* score, float* list, int size, int sizex, int sizey, int numx){
    int tx=threadIdx.x;
    int ty=threadIdx.y;

    tx=blockIdx.x*blockDim.x+tx;
    ty=blockIdx.y*blockDim.y+ty;
    int cx=tx*size;
    int cy=ty*size;


    if(cx<sizex&&cy<sizey){
        int sx=size;
        int sy=size;

        if(cx+size>sizex){
            sx=sizex-cx;
        }
        if(cy+size>sizey){
            sy=sizey-cy;
        }
        score=score+cx*sizex+cy;
        //float max=6.805646932770577*(1000000000000000000000);
        float max=0;
        int maxx=0;
        int maxy=0;
        for(int i=0;i<sx;i++){
            for(int j=0;j<sy;j++){
                if(score[j]>=max){
                    max=score[j];
                    maxx=cx+i;
                    maxy=cy+j;
                }
            }
            score=score+sizex;
        }

        int sub=5*(tx*numx+ty);
        list[sub]=maxx;
        list[sub+1]=maxy;
        list[sub+2]=max;
        list[sub+3]=tx;
        list[sub+4]=ty;
    }
}

__global__ void getMax3(float* score, float* list, int psize, int sizex, int sizey, int numx,int num, int iter){
    int tx=threadIdx.x;
    int ty=threadIdx.y;

    tx=blockIdx.x*blockDim.x+tx;
    ty=blockIdx.y*blockDim.y+ty;
    int sub=(tx*numx+ty)*5;

    if (sub<num){
        int cx=list[sub]-psize/2;
        int cy=list[sub+1]-psize/2;

        float max=0;
        int maxx=0;
        int maxy=0;

        for(int i=0;i<iter;i++){

            if(cx<sizex&&cy<sizey&&cx>0&&cy>0){
                int sx=psize;
                int sy=psize;

                if(cx+sx>sizex){
                    sx=sizex-cx;
                }
                if(cy+sy>sizey){
                    sy=sizey-cy;
                }
                max=0;
                maxx=0;
                maxy=0;
                float* score0=score+cx*sizex+cy;
                for(int i=0;i<sx;i++){
                    for(int j=0;j<sy;j++){
                        if(score0[j]>=max){
                            max=score0[j];
                            maxx=cx+i;
                            maxy=cy+j;
                        }
                    }
                    score0=score0+sizex;
                }
                cx=maxx-psize/2;
                cy=maxy-psize/2;
            }
         }//end for

        list[sub]=maxx;
        list[sub+1]=maxy;
        list[sub+2]=max;
        list[sub+3]=tx;
        list[sub+4]=ty;
    }//end if num
}



__global__ void multiply_them(float *dest, float *a, float *b)
{
  const int i = threadIdx.x;
  dest[i] = a[i] * b[i];
}

""")

class item:
    def __init__(self,count=0,x=0 ,y=0 ):
        self.id=count
        self.minX = x
        self.minY=y
        self.maxX=x
        self.maxY=y
        self.totalN=0
        self.totalS=0
        self.x=x
        self.y=y
        self.p=0.1
    def __str__(self):
        line=str(self.id)+' '+str(self.minX)+' '+str(self.maxX)+' '+str(self.minY)+' '+str(self.maxY) \
                +' '+str(self.totalN)+' '+str(self.totalS)+' '+str(self.x)+' '+str(self.y)+' '+str(self.p)+'\n'
        return line

    def update(self,x,y):
        if(x<self.minX):
            self.minX=x
        elif(x>self.maxX):
            self.maxX=x

        if(y<self.minY):
            self.minY=y
        elif(y>self.maxY):
            self.maxY=y

        self.totalN=self.totalN+1

        return self

    def getS(self):
        width=self.maxX-self.minX+1
        length=self.maxY-self.minY+1
        self.totalS=width*length
        self.x=self.minX+width/2
        self.y=self.minY+width/2
        self.p=float(self.totalN)/self.totalS
        return self.totalS

def reshape(res,num):
    canList=[]
    for i in range(0,num):
        sub=i*5
        m=res[sub+2]
        if m!=0:
            candi=[int(res[sub]),int(res[sub+1]),m,int(res[sub+2]),int(res[sub+3])]
            canList.append(candi)
    print('convert')
    canList.sort(key=lambda x: x[2], reverse=True)

    #for i in range(0, 10):
    #    print(canList[i])
    return canList

def gaussian_kernel_2d_opencv(kernel_size = 3,sigma = 1):
    kx = cv2.getGaussianKernel(kernel_size,sigma)
    ky = cv2.getGaussianKernel(kernel_size,sigma)
    res=np.multiply(kx,np.transpose(ky))
    res=1-(res-np.max(res))/(-1*np.ptp(res))
    return res

def getOverlap(x,y,rx,ry,p):
    t=int(p/2)
    xmin=x-t
    xmax=x+t
    ymin=y-t
    ymax=y+t

    rxmin=rx-t
    rxmax=rx+t
    rymin=ry-t
    rymax=ry+t

    x1=abs(rxmin-xmax)
    x2=abs(rxmax-xmin)
    lx=min(x1,x2)

    y1=abs(rymin-ymax)
    y2=abs(rymax-ymin)
    ly=min(y1,y2)

    return lx*ly

def cleanCanList(canList,op1,psize):
    numCan=len(canList)
    op=op1*psize*psize
    tmp=[-psize,-psize,0]
    for i in range(0,len(canList)):
        candi=canList[i]
        if candi !=tmp:
            x=candi[0]
            y=candi[1]

            for j in range(i+1,len(canList)) :
                candi=canList[j]
                rx=candi[0]
                ry=candi[1]

                if abs(x-rx)<psize and abs(y-ry)<psize:
                    sOp=getOverlap(x,y,rx,ry,psize)
                    if sOp>op:
                        #print(rx,ry,sOp,op)
                        canList[j]=tmp #delete
                        (canList[i])[0]=0.8*x+0.2*rx
                        (canList[i])[1]=0.8*y+0.2*ry


    return [x for x in canList if x != tmp]

def writeStarHead(fstar):
    star=open(fstar,'w')
    star.write('data_\n\n')
    star.write('loop_\n')
    star.write('_rlnCoordinateX #1\n')
    star.write('_rlnCoordinateY #2\n')
    star.write('_rlnParticleSelectZScore  #3\n')
    return star

def writeCan(fstar,canList,sizex, sizey):
    fstar=writeStarHead(fstar)
    p=0
    l=len(canList)
    for i in range(0,l):
        c=canList[i]
        #line=str(c[0])+' '+str(c[1])+' '+str(c[2])+'\n'
        #x=c[1]
        #y=sizey-c[0]
        x=c[0]
        y=c[1]
        x0=y
        y0=x
        if(len(c)>2):
            p=c[2]

        line=str(x0)+' '+str(y0)+' '+str(p)+'\n'
        #print(line)
        fstar.write(line)
    fstar.close()

def reNorm(heatArr, nSep):
    ksize=5

    kernel = np.ones((ksize,ksize),np.uint8)
    img = cv2.erode(heatArr,kernel,iterations = nSep)
    #tmp=int(psize/4)
    #if tmp%2 ==0:
    #    tmp=tmp+1
    #heatArr= cv2.medianBlur(heatArr,tmp)
    #heatArr= cv2.medianBlur(heatArr,5)
    #heatArr= cv2.medianBlur(heatArr,5)
    #heatArr= cv2.medianBlur(heatArr,5)
    return heatArr

def reLev(heatArr, level):
    num=int(255/level)
    heatArr=heatArr/num
    heatArr=heatArr.astype(int)
    return heatArr

def draw(im, cx, cy, sx, sy):
    # Calculate rectangle coordinates
    startx = int(cx - sx / 2)
    starty = int(cy - sy / 2)  # Corrected to use sy
    endx = int(cx + sx / 2)
    endy = int(cy + sy / 2)

    # Calculate colors (simplified for demonstration; adjust as necessary)
    c = 1
    r = (c // 4) * 255
    g = ((c % 4) // 2) * 255
    b = (c % 2) * 255

    # Thickness of the rectangle border
    w = 2

    # Draw the rectangle
    cv2.rectangle(im, (startx, starty), (endx, endy), (b, g, r), w)

    return im

def checkScore(score):
    score0=np.zeros([score.shape[0], score.shape[1],3])
    print(score0.shape, score.shape)
    score0[:,:,0]=score[:,:]
    score0[:,:,1]=score[:,:]
    score0[:,:,2]=score[:,:]
    a=score0
    score0=(255*(a - np.max(a))/-np.ptp(a))
    for c in canList:
        x=int(c[0])
        y=int(c[1])
        sx=sy=psize
        score0=draw(score0,x,y,sx,sy)

def pad_image(image_array):
    # Get the current height and width of the image
    height, width = image_array.shape

    # Find the smallest pixel value in the image to use as padding value
    min_pixel_value = np.min(image_array)

    # Determine the size of padding needed
    if height > width:
        diff = height - width
        padding = ((0, 0), (0, diff))  # Padding only to the right
    else:
        diff = width - height
        padding = ((0, diff), (0, 0))  # Padding only to the bottom

    # Apply padding, using the smallest pixel value found
    padded_image = np.pad(image_array, padding, mode='constant', constant_values=min_pixel_value)
    return padded_image

In [ ]:
nms_list = []
for idx, img in enumerate(label_images):
    # if idx == 6:
    #     break
    heatArr=pad_image(img)
    heatArr=reNorm(heatArr, nSep)
    heatArr=reLev(heatArr,level)
    gaus= gaussian_kernel_2d_opencv(kernel_size = psize,sigma = 0)

    # canList,score=scoreGpuLaunch(heatArr,gaus,psize, gsize, nIter, fscore)
    gsize=psize*pCan
    heat=heatArr.astype(np.float32)
    gaus=gaus.astype(np.float32)
    score=np.zeros(heat.shape).astype(np.float32)
    [sizex,sizey]=heat.shape
    sizex = int(sizex)
    sizey = int(sizey)
    psize = int(psize)
    gsize = int(gsize)

    func = mod.get_function("scoreGpu")

    tx=16
    ty=16
    bx=(sizex-1)//tx+1
    by=(sizey-1)//ty+1
    print('get score gpu',tx, ty, bx, by, gsize, psize)


    func(drv.In(heat), drv.In(gaus), drv.Out(score), np.int32(sizex), np.int32(sizey), np.int32(psize),
        block=(tx, ty, 1), grid=(int(bx), int(by)))

    # # Calculate necessary dimensions and convert all to int
    numx = int((sizex - 1) // gsize + 1)
    numy = int((sizey - 1) // gsize + 1)
    num = numx * numy
    tnum = num * 5

    # # Create the result array
    res = np.zeros(tnum).astype(np.float32)

    # # Block and grid dimensions
    bx = (numx - 1) // tx + 1
    by = (numy - 1) // ty + 1

    # Get the function from the module
    func = mod.get_function("getMax")

    # Call the function, ensure all parameters are the correct type
    func(drv.In(score), drv.Out(res), np.int32(gsize), np.int32(sizex), np.int32(sizey), np.int32(numx),
        block=(16, 16, 1), grid=(bx, by, 1))

    # Make sure all dimensions and sizes are properly cast to np.int32 to avoid ambiguity
    niter = np.int32(nIter)
    gsize = np.int32(gsize)
    sizex = np.int32(sizex)
    sizey = np.int32(sizey)
    numx = np.int32(numx)
    tnum = np.int32(num)

    print('get Max3', tx, ty, bx, by, niter, 'other : ', gsize, sizex, sizey, numx, tnum)
    func = mod.get_function("getMax3")
    # Ensuring the correct parameter order and type for the kernel invocation
    func(drv.In(score), drv.InOut(res), gsize, sizex, sizey, numx, tnum, niter,
            block=(16, 16, 1), grid=(bx, by, 1))

    canList=reshape(res,num)

    print('Number of Particles before:', len(canList))
    if(len(canList)>pnum):
        canList=canList[:pnum]


    canList=cleanCanList(canList,overlap,psize)
    #canList=reCan(canList,ratio)
    print('Number of Particles:', len(canList))
    nms_list.append([(r[1],r[0]) for r in canList])

get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 2347
Number of Particles: 92
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 1444
Number of Particles: 173
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 573
Number of Particles: 97
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 2914
Number of Particles: 132
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 3044
Number of Particles: 146
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838 63 3969
convert
Number of Particles before: 3076
Number of Particles: 150
get score gpu 16 16 240 240 61 154
get Max3 16 16 4 4 100 other :  61 3838 3838

In [ ]:
import matplotlib
for idx, (test_image, _, grid, _) in enumerate(full_dataset):
    # if idx == 6:
    #     break
    if idx % 30 == 0:
    # else:
        label_image = label_images[idx]
        _, ax = plt.subplots(figsize=(12, 12))
        ax.imshow(processed_micrographs[idx], cmap='gray')
        ax.imshow(label_image, cmap='inferno', alpha=0.4)

        for indx in nms_list[idx]:
            x, y = indx
            c = matplotlib.patches.Circle((x, y), radius=RADIUS, fill=False, color='r')
            ax.add_patch(c)
        plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import pandas as pd

for idx, fname in enumerate(fnames):
    # Create the adjusted list of tuples for current index
    adjusted_c_list = [(x + BORDER, y + BORDER) for x, y in nms_list[idx]]

    # Create a temporary DataFrame from the list of tuples
    temp_df = pd.DataFrame(adjusted_c_list, columns=['rlnCoordinateX', 'rlnCoordinateY'])
    temp_df['rlnImageName'] = [f"{i:05d}@{fname}.mrc" for i in range(len(temp_df))]
    temp_df['rlnMicrographName'] = fname + ".mrc"

    # If this is the first DataFrame, initialize df
    if idx == 0:
        df = temp_df
    else:
        # Append the temporary DataFrame to the existing df
        df = pd.concat([df, temp_df], ignore_index=True)

In [ ]:
df

,rlnCoordinateX,rlnCoordinateY,rlnImageName,rlnMicrographName
0,2831.8,2374.2,00000@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
1,413.0,622.0,00001@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
2,3044.4,1630.2,00002@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
3,349.0,285.0,00003@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
4,1700.0,2001.0,00004@HCN1apo_0030_2xaligned.mrc,HCN1apo_0030_2xaligned.mrc
...,...,...,...,...
45318,2351.0,3233.0,00199@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc
45319,121.2,2048.0,00200@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc
45320,3611.0,2712.0,00201@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc
45321,3577.0,2576.0,00202@HCN1apo_1060_2xaligned.mrc,HCN1apo_1060_2xaligned.mrc


In [ ]:
starfile.write(df, f'{RESULT_DIR}/nms_particles.star')